In [1]:
import numpy as np
import pandas as pd
from tsfresh import feature_extraction
import os
import seaborn as sns
import re
from collections import defaultdict

ImportError: cannot import name 'feature_extraction' from 'tsfresh' (unknown location)

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

In [ ]:
def normalize_signal(signal):
    """
    time_series
    
    Returns normalized TS: (TS - mean(TS))/std(TS)
    """
    std = np.std(signal)
    norm_signal = (signal - np.mean(signal)) / std if std else 1
    return norm_signal

In [ ]:
def normalize_all_signals_in_ds(one_group_ds):
    for signal in one_group_ds.drop('t', axis=1):
        one_group_ds[signal] = normalize_signal(one_group_ds[signal])

In [ ]:
# os.chdir('..')
os.getcwd()

In [ ]:
# labels = pd.DataFrame(columns=['id', 'class', 'label'])
# labels.columns
# labels.append((pd.Series( [1, 2, 3], index=labels.columns)), ignore_index=True)
# # dfObj.append(pd.Series(['Raju', 21, 'Bangalore', 'India'], index=dfObj.columns ), ignore_index=True)

In [ ]:
def append_signals_dataset(data_directory, signals_dataset = None, labels = None):
    data_dir_list = os.listdir(data_directory)
    os.chdir(data_directory)
    signal_id = 0
    if not labels:
        labels = pd.DataFrame(columns=['id', 'class', 'label'])
    for signal_label, directory in enumerate(data_dir_list):
        if os.path.isfile(directory) or directory[0] == '.':
                continue
        os.chdir(directory)
        for filename in os.listdir(os.curdir):
            if re.match(r'(.*csv)', filename):
                if (signals_dataset) is not None:
                    tmp_df = pd.read_csv(filename)
                    normalize_all_signals_in_ds(tmp_df)
                    tmp_df['id'] = signal_id
                    signals_dataset = signals_dataset.append(tmp_df, ignore_index=True)
                else:
                    signals_dataset = pd.read_csv(filename)
                    normalize_all_signals_in_ds(signals_dataset)
                    signals_dataset['id'] = 0
#                 print(signals_dataset.head(5))
                tmp_labels = pd.Series([signal_id, directory, np.int(signal_label)],
                                         index=labels.columns)
                labels = labels.append(tmp_labels, ignore_index=True)
                signal_id += 1
        os.chdir('../')
    os.chdir('../..')
    return signals_dataset, labels

Data preprocessing:
    - dropping U.1 == U
    - changing type
    - generating features

In [ ]:
# Dataset for first gen
dataset, labels = append_signals_dataset('./data/gen_1_25')
# Dataset for first gen
dataset_2, labels_2 = append_signals_dataset('./data/gen_3_80')

In [ ]:
def normalize_labels(labels_df):
    """
    Разметим все классы числами от 0 до количества классов
    и преобразуем новую колонку в формат чисел (float)
    """
    unique_classes = np.unique(labels_df['class'])
    labels_df['label'] = labels['class'].map(dict(zip(unique_classes, range(len(unique_classes)))))
    labels_df.label = pd.to_numeric(labels.label)

In [ ]:
labels.shape

In [ ]:
print("Уникальные классы в датасете:", np.unique(labels['class']))
normalize_labels(labels)
normalize_labels(labels_2)
labels.sample(5)

In [ ]:
# Удалим колонки которые не несут достаточной информации (повторяют значения, не изменяются)
# или могут быть некорректны
def drop_columns(dataset):
    mod_dataset = dataset.drop(['U.1', 'Pgt', 'Pij'], axis=1)
    return mod_dataset
mod_dataset = drop_columns(dataset)
mod_dataset_2 = drop_columns(dataset_2)
# Проверим возможность склеить сигнал с метками
mod_dataset.merge(labels, on='id')

In [ ]:
features = defaultdict()
for param in mod_dataset.loc[:,'U':'delta']:
    features[param] = feature_extraction.extract_features\
    (mod_dataset.loc[:,[param,'id','t']], column_sort='t', column_id='id')

Извлечем те же признаки для проверочного датасета, который модель никогда не видела

In [ ]:
features_val = defaultdict()
for param in mod_dataset_2.loc[:,'U':'delta']:
    features_val[param] = feature_extraction.extract_features\
    (mod_dataset_2.loc[:,[param,'id','t']], column_sort='t', column_id='id')

### После обработки каждого сигнала библиотекой tsfresh извлекаем всевозможные признаки, признаки классифицированы по сигналу, из которого были получены (словарь features)
В конечном счете для каждого сигнала сформирован следующий набор данных:

In [ ]:
for param in features:
    print("Для параметра", param, "получено", features[param].shape[1], "признака(ов).")

In [ ]:
# Пример хранящихся признаков
features['U'].columns

## Поиск наиболее важных признаков, которые позволят предсказывать результат переходного процесса

In [ ]:
# Удаляем фичи количество признаков в которых меньше количества классов
def clear_useless_features(features, threshold=2):
    for feature in features.columns:
        if features[feature].nunique() <= threshold:
            features = features.drop(feature, axis=1)
    return features

In [ ]:
modified_features = defaultdict()
for param in features:
    modified_features[param] = clear_useless_features(features[param],
                                                     labels.label.nunique())
    print(f"Число удаленных признаков для {param}:",
          features[param].shape[1] - modified_features[param].shape[1])

Поиск признаков, имеющих наибольшую корреляцию с целевой переменной sklearn (Только для регресии и неотрицательных признаков)

In [ ]:
def delete_negative_features(X):
    for i in X.columns:
        if X[i].min() < 0:
            X = X.drop(i, axis=1)
    return X

In [ ]:
X = delete_negative_features(modified_features['U'])
X_2 = 
y = labels.label
# bestfeatures = SelectKBest(score_func=chi2, k=10)
# fit = bestfeatures.fit(X, labels.label)
# dfscores = pd.DataFrame(fit.scores_)
# dfcolumns = pd.DataFrame(X.columns)
# featureScores = pd.concat([dfcolumns,dfscores],axis=1)
# featureScores.columns = ['Specs','Score']  #naming the dataframe columns
# print(featureScores.nlargest(10,'Score'))

Использование дерева для поиска фич

In [ ]:
def plot_features(X, Y):
#     X = modified_features['U']
#     y = labels.label
    model = ExtraTreesClassifier(n_estimators=200)
    model.fit(X,y)
#     print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
    #plot graph of feature importances for better visualization
    feat_importances = pd.Series(model.feature_importances_, index=X.columns)
    feat_importances.nlargest(15).plot(kind='barh')
    plt.show()
    return feat_importances.nlargest(15)

In [ ]:
most_valuable_feat = dict([[feat, None] for feat in features])
for param in features:
    most_valuable_feat[param] = (plot_features(modified_features[param], labels.label))

In [ ]:
# Создаем датафрэйм с индексами, который будем доплнять самыми релевантными фичами
def remain_relevant_features(features):
    resulted_df = pd.DataFrame(index=features['U'].index)
    # Дополняем датафрэйм
    for param in features:
        for feat in features[param]:
            if feat in most_valuable_feat[param]:
                if most_valuable_feat[param][feat] >= 0.015:
                    resulted_df = resulted_df.join(features[param][feat])
    return resulted_df

In [ ]:
# Выведем первые две строки получившегося датафрэйма
resulted_df = remain_relevant_features(features)
resulted_df.head(2)

In [ ]:
print("Оставшиеся фичи:")
print(*resulted_df.columns, sep='\n')

Построим тепловую карту фич, взятых из предыдущего шага

In [ ]:
# data = features['U'].iloc[:,0:50].join(labels.loc[:,['label']], on='id')

In [ ]:
## get correlations of each features in dataset
data = resulted_df.join(labels.loc[:,['id', 'label']])
corrmat = data.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(data[top_corr_features].corr(),annot=True,cmap="RdYlGn")

## Постоение простейших моделей, градиентный бустинг и ансамбль решающих деревьев

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(resulted_df, labels.label, test_size=0.2, random_state=42)

### Ансамбль решающих деревьев

In [ ]:
clf = RandomForestClassifier(n_estimators=200, max_depth=5, max_features=0.7)
scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1')
scores

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict_proba(X_test)
y_scores = y_pred[:,1]
y_scores

In [ ]:
y_true = np.array(y_test)
metrics.roc_auc_score(y_true, y_scores)

## Проверка на другом генераторе, который модель никогда ранее не видела

In [ ]:
resulted_df_val = remain_relevant_features(features_val)
labels_2.label = pd.to_numeric(labels_2.label)
X_val, y_val = resulted_df_val, labels_2.label
y_pred = clf.predict_proba(X_val)
y_scores = y_pred[:,1]
y_predicted_class = clf.predict(X_val)
print(y_scores)
y_predicted_class

In [ ]:
y_true = np.array(y_val)
y_true

In [ ]:
print("ROC AUC score:", metrics.roc_auc_score(y_true, y_scores))
print("F1 score:", metrics.f1_score(y_true, y_predicted_class))

### Градиентный бустинг

In [ ]:
clf = GradientBoostingClassifier(learning_rate=0.01, random_state=42)
scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1')
scores

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict_proba(X_test)
y_scores = y_pred[:,1]
y_scores

In [ ]:
y_true = np.array(y_test)
metrics.roc_auc_score(y_true, y_scores)

## Проверка градиентного бустинга

In [ ]:
resulted_df_val = remain_relevant_features(features_val)
labels_2.label = pd.to_numeric(labels_2.label)
X_val, y_val = resulted_df_val, labels_2.label
y_pred = clf.predict_proba(X_val)
y_scores = y_pred[:,1]
y_predicted_class = clf.predict(X_val)
y_scores

In [ ]:
y_true = np.array(y_val)
y_true

In [ ]:
print("ROC AUC score:", metrics.roc_auc_score(y_true, y_scores))
print("F1 score:", metrics.f1_score(y_true, y_predicted_class))

Гит комит

In [ ]:
!git status
!git add .
!git commit -m "Added another generator"
